In [353]:
import urllib.parse
level = "ARTICLE"
date = "[18500000 TO 18600000]"
art_type = "Classified ads"
offset = "0"
limit = "500"
query = "<"+level+"§articleType:\""+art_type+"\" AND dateStart:"+date+"§"+level+">"
query = urllib.parse.quote(query)
#url = "https://vm0824.kaj.pouta.csc.fi/octavo/bln1800/search??pretty&fieldEnricher=&offsetDataConverter=&query="
url = "https://vm0824.kaj.pouta.csc.fi/octavo/bln1800/queryStats?query="
url += query
url+= "&field=newspaperID&maxDocs=-1"
#url += "&field=newspaperID&field=ocrConfidence&field=text&field=title&offset="+offset
#url += "&limit="+limit
#url += "&snippetLimit=0&contextLevel=Sentence&contextExpandLeft=0&contextExpandRight=0&level=&endpoint=https%3A%2F%2Fvm0824.kaj.pouta.csc.fi%2Foctavo%2Fbln1800%2F"
print(url)

https://vm0824.kaj.pouta.csc.fi/octavo/bln1800/queryStats?query=%3CARTICLE%C2%A7articleType%3A%22Classified%20ads%22%20AND%20dateStart%3A%5B18500000%20TO%2018600000%5D%C2%A7ARTICLE%3E&field=newspaperID&maxDocs=-1


In [354]:
import urllib.request, json 
import re
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd

import pyLDAvis
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
pd.options.mode.chained_assignment = None

In [355]:
#Importing Gensim and it's dependencies
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim 
from gensim import models, corpora

In [356]:
stop_words = stopwords.words('english')

In [357]:
with urllib.request.urlopen(url) as url:
    data = json.loads(url.read().decode())

In [358]:
paper_id = []
for k in range(total_papers):
              paper_id.append(data['results']['grouped'][k]['fields']['newspaperID'])

In [359]:
len(set(paper_id))

122

In [362]:
newspaper_id = 'BNCN0013'
level = "ARTICLE"
date = "[18500000 TO 18510000]"
art_type = "Classified ads"
offset = "0"
limit = "-1"
query = "<"+level+"§articleType:\""+art_type+"\" AND dateStart:"+date+" AND newspaperID:\""+newspaper_id+"\"§"+level+">" 
query = urllib.parse.quote(query)
url = "https://vm0824.kaj.pouta.csc.fi/octavo/bln1800/search??pretty&fieldEnricher=&offsetDataConverter=&query="
url += query
#url += "&field=newspaperID&field=ocrConfidence&field=text&field=title&offset="+offset
url+= "&field=ocrConfidence&field=text&field=title&offset="+offset
url += "&limit="+limit
url += "&snippetLimit=0&contextLevel=Sentence&contextExpandLeft=0&contextExpandRight=0&level=&endpoint=https%3A%2F%2Fvm0824.kaj.pouta.csc.fi%2Foctavo%2Fbln1800%2F"
print(url)

https://vm0824.kaj.pouta.csc.fi/octavo/bln1800/search??pretty&fieldEnricher=&offsetDataConverter=&query=%3CARTICLE%C2%A7articleType%3A%22Classified%20ads%22%20AND%20dateStart%3A%5B18500000%20TO%2018510000%5D%20AND%20newspaperID%3A%22BNCN0013%22%C2%A7ARTICLE%3E&field=ocrConfidence&field=text&field=title&offset=0&limit=-1&snippetLimit=0&contextLevel=Sentence&contextExpandLeft=0&contextExpandRight=0&level=&endpoint=https%3A%2F%2Fvm0824.kaj.pouta.csc.fi%2Foctavo%2Fbln1800%2F


In [363]:
with urllib.request.urlopen(url) as url:
    data = json.loads(url.read().decode())

In [364]:
data['results']['docs'][499].keys()

dict_keys(['text', 'title', 'ocrConfidence', 'score'])

In [365]:
len(data['results']['docs'])

910

In [294]:
total_articles = len(data['results']['grouped'])
total_articles

122

In [252]:
data['results']['docs'][0].keys()

dict_keys(['text', 'newspaperID', 'title', 'ocrConfidence', 'score'])

In [253]:
paper_id = []
ads= []
for i in range(0, total_articles):
    ads.append(data['results']['docs'][i]['text'])
    paper_id.append(data['results']['docs'][i]['newspaperID'])
    

In [254]:
data['results']['docs'][0]

{'text': 'WAHTED,\n\nIN . small Family, where two Servants are kept, a\n\nGOOD PLAIN COOK, who must have an excel-\n\nlent character.\n\nApp\'y t0 tne PaiNTER.\n\nWANTED,\n\n\\S HOUSEMAID, a YOUNG WOMAN, who must\n\nbring with her a good Character.\n\nApply to Mrs. HOPWOOD, Rock Cliffe.\n\nWANTED,\n\nIK a Power-Loom Shirting Manufactory, a MAN\n\nwbo has a thorough knowledge of the weaving\n\ndepartment, and can take the general management :\ncone need apply who have not filled a similar situation.\n\nAddress Box 303, Post-Office, Manchester.\n\nLOWER BANK ACADEMY~\n\nJ. THOMPSON\n\nRESPECTFULLY informs hi. Friend, that the du-\n\ntie. of hi. Establishment will be resumed on the\n\n15th inst.\n\nDANCING.\n\nMR. G ILLINS\n\nRESUMES his Instruction for the ADULT CLASS\n\non FRIDAY EVENING NEXT, at the Bull Inn.\n\nJanuary 7th.\n\nTHE SIXTH\n\nANNUAL BALL\n\nIN aid of the Funds of the BLACKBURN MECHA-\n\nNICS\' INSTITUTION, will be held in the AS-\n\nSEMBLY ROOM, Heaton Street, on MONDAY,

In [326]:
set(paper_id)

{'BNCN0002',
 'BNCN0004',
 'BNCN0005',
 'BNCN0010',
 'BNCN0013',
 'BNCN0014',
 'BNCN0015',
 'BNCN0016',
 'BNCN0017',
 'BNCN0021',
 'BNCN0024',
 'BNCN0026',
 'BNCN0027',
 'BNCN0030',
 'BNCN0035',
 'BNCN0039',
 'BNCN0049',
 'BNCN0051',
 'BNCN0053',
 'BNCN0056',
 'BNCN0063',
 'BNCN0066',
 'BNCN0070',
 'BNCN0071',
 'BNCN0073',
 'BNCN0078',
 'BNCN0079',
 'BNCN0080',
 'BNCN0081',
 'BSNP0193',
 'BSNP0196',
 'BSNP0197',
 'BSNP0205',
 'BSNP0207',
 'BSNP0218',
 'BSNP0221',
 'BSNP0224',
 'BSNP0225',
 'BSNP0226',
 'BSNP0228',
 'BSNP0237',
 'BSNP0243',
 'BSNP0244',
 'BSNP0247',
 'BSNP0249',
 'BSNP0250',
 'BSNP0257',
 'BSNP0264',
 'BSNP0265',
 'BSNP0266',
 'BSNP0267',
 'BSNP0270',
 'BSNP0284',
 'BSNP0298',
 'BSNP0309',
 'BSNP0312',
 'BSNP0315',
 'BSNP0317',
 'BSNP0328',
 'BSNP0330',
 'BSNP0336',
 'BSNP0342',
 'BSNP0343',
 'BSNP0344',
 'BSNP0350',
 'BSNP0352',
 'BSNP0358',
 'BSNP0360',
 'BSNP0361',
 'BSNP0363',
 'BSNP0366',
 'BSNP0369',
 'BSNP0384',
 'BSNP0396',
 'BSNP0398',
 'BSNP0399',
 'BSNP0400',

In [132]:
cleaned_text = [t for t in ads if t not in stop_words and re.match(r'[a-zA-Z\-][a-zA-Z\-]{3,}', t)]

In [82]:
text_string = ""
new_pattern = '[A-Za-z]+'
for line in cleaned_text:
    for item in re.finditer(new_pattern, line):
        text_string+=(item.group()+" ")

In [83]:
final_text = word_tokenize(str(text_string).lower())

In [84]:
word = 'NOTICES'
if word in final_text: 
    print ('success')

In [352]:
string = final_text
substring = "pills"
count = string.count(substring)
count

105

In [301]:
len(text_string.split(' '))

296769

In [302]:
count/len(text_string.split(' '))

0.038979812581502786

In [295]:
len('pills')

5